[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2022/blob/main/40--spark/43--jobs-playground-one-more-time.ipynb)

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz -O spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz

--2023-01-14 05:46:43--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M   138MB/s    in 2.1s    

2023-01-14 05:46:45 (138 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [3]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.7 MB/s eta 0:00:00


In [4]:
!mkdir -p var/
!wget -nc https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/jobs.zip
!unzip -u jobs.zip -d jobs_dataset

--2023-01-14 05:47:34--  https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/jobs.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.140.115
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.140.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11339888 (11M) [application/zip]
Saving to: ‘jobs.zip’

jobs.zip            100%[===================>]  10.81M  9.91MB/s    in 1.1s    

2023-01-14 05:47:36 (9.91 MB/s) - ‘jobs.zip’ saved [11339888/11339888]

Archive:  jobs.zip
  inflating: jobs_dataset/0001.json  
  inflating: jobs_dataset/0002.json  
  inflating: jobs_dataset/0003.json  
  inflating: jobs_dataset/0004.json  
  inflating: jobs_dataset/0005.json  
  inflating: jobs_dataset/0006.json  
  inflating: jobs_dataset/0007.json  
  inflating: jobs_dataset/0008.json  
  inflating: jobs_dataset/0009.json  
  inflating: jobs_dataset/0010.json  
  inflating: jobs_dataset/0011.json  
  

In [5]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
        .master("local")
        .appName("Jobs playground")
        .config('spark.ui.port', '4050')
        .getOrCreate())

In [6]:
df = spark.read.json("jobs_dataset/")

In [7]:
df.printSchema()

root
 |-- apply: struct (nullable = true)
 |    |-- onlineInterviewAvailable: boolean (nullable = true)
 |    |-- url: string (nullable = true)
 |-- basics: struct (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- seniority: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- technology: string (nullable = true)
 |-- benefits: struct (nullable = true)
 |    |-- benefits: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- equipment: struct (nullable = true)
 |    |    |-- computer: string (nullable = true)
 |    |    |-- monitors: string (nullable = true)
 |    |    |-- operatingSystems: struct (nullable = true)
 |    |    |    |-- lin: boolean (nullable = true)
 |    |    |    |-- mac: boolean (nullable = true)
 |    |    |    |-- win: boolean (nullable = true)
 |    |-- officePerks: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- city: string (nullable = true

In [8]:
df.show(3)

+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------+---------------+----+--------------------+------+--------------------+-----------+------------------+-------------+--------------------+-----------------+-------+-------+--------------------+--------------------+--------------------+---------+--------------------+---------+--------------------+-------------------+
|               apply|              basics|            benefits|city|             company|          companyUrl|            consents|             details|          essentials|highlightedMsg|      id|invalidSections|link|            location|  meta|         methodology|oneTimeEdit|postHighlightedMsg|       posted|          postingUrl|preHighlightedMsg|preview|regions|        requirements|  sectionsValidation|                 seo|simillars|               specs|   status|technologyOrCa

In [9]:
df.createOrReplaceTempView("jobs")

In [10]:
spark.sql("""
Select * from jobs          
""").show(3)

+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------+---------------+----+--------------------+------+--------------------+-----------+------------------+-------------+--------------------+-----------------+-------+-------+--------------------+--------------------+--------------------+---------+--------------------+---------+--------------------+-------------------+
|               apply|              basics|            benefits|city|             company|          companyUrl|            consents|             details|          essentials|highlightedMsg|      id|invalidSections|link|            location|  meta|         methodology|oneTimeEdit|postHighlightedMsg|       posted|          postingUrl|preHighlightedMsg|preview|regions|        requirements|  sectionsValidation|                 seo|simillars|               specs|   status|technologyOrCa

In [13]:
CityAwareDF = spark.sql("select * from jobs where city is not null")

+-----+------+--------+------------+-------+----------+--------+-------+----------+--------------+----+---------------+--------------------+--------+----+-----------+-----------+------------------+------+----------+-----------------+-------+-------+------------+------------------+----+--------------------+-----+-------+--------------------+-----+
|apply|basics|benefits|        city|company|companyUrl|consents|details|essentials|highlightedMsg|  id|invalidSections|                link|location|meta|methodology|oneTimeEdit|postHighlightedMsg|posted|postingUrl|preHighlightedMsg|preview|regions|requirements|sectionsValidation| seo|           simillars|specs| status|technologyOrCategory|title|
+-----+------+--------+------------+-------+----------+--------+-------+----------+--------------+----+---------------+--------------------+--------+----+-----------+-----------+------------------+------+----------+-----------------+-------+-------+------------+------------------+----+----------------

In [14]:
import pyspark.sql.functions as F
CityStartsWithK = CityAwareDF.select('*').where(CityAwareDF['city']=='krakow')

In [16]:
CityStartsWithK.show(1)

+-----+------+--------+------+-------+----------+--------+-------+----------+--------------+----+---------------+-----------------+--------+----+-----------+-----------+------------------+------+----------+-----------------+-------+-------+------------+------------------+----+--------------------+-----+-------+--------------------+-----+
|apply|basics|benefits|  city|company|companyUrl|consents|details|essentials|highlightedMsg|  id|invalidSections|             link|location|meta|methodology|oneTimeEdit|postHighlightedMsg|posted|postingUrl|preHighlightedMsg|preview|regions|requirements|sectionsValidation| seo|           simillars|specs| status|technologyOrCategory|title|
+-----+------+--------+------+-------+----------+--------+-------+----------+--------------+----+---------------+-----------------+--------+----+-----------+-----------+------------------+------+----------+-----------------+-------+-------+------------+------------------+----+--------------------+-----+-------+--------

In [19]:
links = CityStartsWithK.select('*').rdd.map(lambda row: row.link)

In [ ]:
links.take(10)

In [ ]:
spark.stop()